# Trabalho de Análise de Rede Complexas.

## Imports

In [1]:
import requests
import gzip
import io
import json
import networkx as nx
from datetime import datetime
import matplotlib.pyplot as plt

## 1. Seleção/Coleta de Dados

- Coletando dados de arquivos do github.

In [2]:
DATABASE_URL = "https://data.gharchive.org/2023-06-10-0.json.gz"

def download_and_parse(url):

    r = requests.get(url)
    r.raise_for_status()

    compressed_file = io.BytesIO(r.content)

    with gzip.GzipFile(fileobj=compressed_file) as f:
        for line in f:
            yield json.loads(line)

- Função para construir o grafo

In [3]:
def build_graph(events):
    G = nx.Graph()
    repo_to_users = {}

    for e in events:
        if e.get("type") == "PushEvent":
            repo_name = e["repo"]["name"]
            user = e["actor"]["login"]

            if repo_name not in repo_to_users:
                repo_to_users[repo_name] = set()
            repo_to_users[repo_name].add(user)

    for repo, users in repo_to_users.items():
        users_list = list(users)

        for i in range(len(users_list)):
            for j in range(i+1, len(users_list)):
                u1 = users_list[i]
                u2 = users_list[j]

                if G.has_edge(u1, u2):
                    G[u1][u2]['weight'] += 1
                else:
                    G.add_edge(u1, u2, weight=1)
    return G


- Função para desenhar o grafo

In [4]:
def draw_graph(graph, degrees):
    plt.figure(figsize=(12, 12))

    pos = nx.spring_layout(graph, k=0.1)

    nx.draw_networkx_nodes(graph, pos, node_size=[degrees[n] for n in graph.nodes()], node_color="blue", alpha=0.7)
    nx.draw_networkx_edges(graph, pos, alpha=0.3)
    nx.draw_networkx_labels(graph, pos, font_size=8)

- Função para carregar os graus

In [5]:
def load_degrees(graph):
    degrees = dict(graph.degree())

    degree_to_users = {}

    for user, degree in degrees.items():
        if degree not in degree_to_users:
            degree_to_users[degree] = []

        degree_to_users[degree].append(user)

    return degree_to_users, degrees

## 2. Definição das questões de pesquisa

1. Quais desenvolvedores colaboram com mais frequência com outros em projetos públicos?
2. Existem grupos de desenvolvedores que colaboram preferencialmente entre si?
3. Qual a densidade da rede de colaboração e o grau médio entre desenvolvedores?
4. Existem desenvolvedores que funcionam como “ponte” entre grupos distintos?
5. Quais repositórios promovem mais colaboração entre usuários diferentes?

## 3. Visualizações da rede

- Carregando o grafo e seus graus

In [ ]:
events = download_and_parse(DATABASE_URL)

graph = build_graph(events)

degrees_to_user, degrees = load_degrees(graph)

### 1ª Visualização - Componentes conexos
- Isola e destaca o maior grupo de desenvolvedores conectados entre si.

In [ ]:
largest_cc = max(nx.connected_components(graph), key=len)
subgraph = graph.subgraph(largest_cc)

draw_graph(subgraph, degrees)
plt.title("Maior componente conexo da rede")
plt.savefig("./assets/maior_componente.png")

### 2ª Visualização - Subgrafo com os 20 nós mais conectados

In [ ]:
top_users = sorted(degrees.items(), key=lambda x: x[1], reverse=True)[:20]
top_usernames = [u for u, _ in top_users]
subgraph = graph.subgraph(top_usernames)

draw_graph(subgraph, degrees)
plt.title("Subgrafo: Top 20 desenvolvedores por grau")
plt.savefig("./assets/top20_grafo.png")

### 3ª Visualização - Rede com nós dimensionados pela centralidade (grau)

In [ ]:
draw_graph(graph, degrees)

plt.title("Rede de Colaboração entre Desenvolvedores (PushEvent)")
plt.axis("off")
plt.tight_layout()
plt.savefig('./assets/rede-colaboracao-devs.png')

## 4. Análise estrutural

## 5. Aplicação de técnicas adicionais

## 6. Discussão e conclusões